In [2]:
import pandas as pd 
df = pd.read_csv('../data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()

In [6]:
X = df[['Pclass', 'Cabin', 'Sex', 'Age']]

In [15]:
X.Age.isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Name: Age, Length: 891, dtype: bool

In [22]:
xmean = X.Age.mean()

In [27]:
X[X.Age.isnull()].Age = xmean 

/Users/t_ishikawa/workspace/python_work/my-python-scripts-and-notebooks/venv/lib/python3.7/site-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [28]:
X.loc[X.Age.isnull(), 'Age'] = xmean

/Users/t_ishikawa/workspace/python_work/my-python-scripts-and-notebooks/venv/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [31]:
y = df.Survived

In [33]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [37]:
X.Cabin = X.Cabin.astype(str) 

/Users/t_ishikawa/workspace/python_work/my-python-scripts-and-notebooks/venv/lib/python3.7/site-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [42]:
encoded_cabin= label_encoder.fit_transform(X.Cabin)

In [49]:
encoded_sex = label_encoder.fit_transform(X.Sex)
encoded_class = label_encoder.fit_transform(X.Pclass)
train_features  = pd.DataFrame([encoded_class, encoded_class, encoded_sex, X.Age]).T 

In [50]:
dtree.fit(train_features, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [51]:
train_features.head()

,0,1,2,3
0,2.0,2.0,1.0,22.0
1,0.0,0.0,0.0,38.0
2,2.0,2.0,0.0,26.0
3,0.0,0.0,0.0,35.0
4,2.0,2.0,1.0,35.0


In [52]:
preds = dtree.predict(X=train_features)

In [53]:
dtree.score(X=train_features, y=y)

0.8799102132435466

In [54]:
from sklearn import metrics
metrics.confusion_matrix(y_true=y, y_pred=preds)

array([[518,  31],
       [ 76, 266]])

In [57]:
print(metrics.classification_report(y_true=y, y_pred=preds))

              precision    recall  f1-score   support

           0       0.87      0.94      0.91       549
           1       0.90      0.78      0.83       342

    accuracy                           0.88       891
   macro avg       0.88      0.86      0.87       891
weighted avg       0.88      0.88      0.88       891



In [58]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [59]:
lr.fit(train_features, y)

/Users/t_ishikawa/workspace/python_work/my-python-scripts-and-notebooks/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
lr_preds = lr.predict(train_features)

In [61]:
metrics.confusion_matrix(y_true=y, y_pred=lr_preds)

array([[472,  77],
       [106, 236]])

In [62]:
print(metrics.classification_report(y_true=y, y_pred=lr_preds))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       549
           1       0.75      0.69      0.72       342

    accuracy                           0.79       891
   macro avg       0.79      0.77      0.78       891
weighted avg       0.79      0.79      0.79       891

